# Tutorial: X-ray image processing

This tutorial demonstrates how to read and process X-ray images with NumPy, imageio, matplotlib and SciPy. You will learn how to read medical images, focus on certain parts, and visually compare them. X-ray image analysis can be part of your data analysis and [machine learning](https://www.sciencedirect.com/science/article/pii/S235291481930214X) workflow when, for example, you're building an algorithm that helps [detect pneumonia (on Kaggle)](https://www.kaggle.com/c/rsna-pneumonia-detection-challenge). In the healthcare industry, medical image processing and analysis is particularly important when images are estimated to account for [at least 90%](https://www-03.ibm.com/press/us/en/pressrelease/51146.wss) of all medical data.

You'll be working with radiology images from the [ChestX-ray8](https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community
) dataset provided by the [National Institutes of Health (NIH)](http://nih.gov). ChestX-ray8 contains over 100,000 de-identified X-ray images in the PNG format from more than 30,000 patients. You can find ChestX-ray8's files on NIH's public Box [repository](https://nihcc.app.box.com/v/ChestXray-NIHCC) in the [`/images`](https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/37178474737) folder. (For more details, refer to the research [paper](http://openaccess.thecvf.com/content_cvpr_2017/papers/Wang_ChestX-ray8_Hospital-Scale_Chest_CVPR_2017_paper.pdf) published at CVPR — a computer vision conference — in 2017.)

For your convenience, a small number of PNG images have been saved to this tutorial's repository under `/biomedical-images`, since ChestX-ray8 contains gigabytes of data and you may find it challenging to download it in batches.

<center><img src="tutorial-x-ray-image-processing.png", width="1000", hspace="20" vspace="20"></center>


### Prerequisites

The reader should have some knowledge of Python, NumPy arrays, and matplotlib. To refresh the memory, you can take the [Python](https://docs.python.org/dev/tutorial/index.html) and matplotlib [PyPlot](https://matplotlib.org/tutorials/introductory/pyplot.html) tutorials, and the NumPy [quickstart](https://numpy.org/devdocs/user/quickstart.html). You can also refer to Scipy Lecture Notes' [Image manipulation and processing using Numpy and Scipy](https://scipy-lectures.org/advanced/image_processing/index.html) tutorial.

The following packages are used in this tutorial:

- [imageio](https://imageio.github.io) for reading and writing image data. The healthcare industry usually works with the [DICOM](https://en.wikipedia.org/wiki/DICOM) format for medical imaging and [imageio](https://imageio.readthedocs.io/en/stable/format_dicom.html) should be well-suited for reading that format. In this tutorial, however, you'll be working with PNG files.
- [matplotlib](https://matplotlib.org/) for data visualization.
- [SciPy](https://www.scipy.org) for multi-dimensional image processing via [`ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html).

This tutorial can be run locally in an isolated environment, such as [Virtualenv](https://virtualenv.pypa.io/en/stable/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html). You can use [Jupyter Notebook or JupyterLab](https://jupyter.org/install) to run each notebook cell. Don't forget to set up [NumPy](https://numpy.org/doc/stable/user/absolute_beginners.html#installing-numpy), [matplotlib](https://matplotlib.org/users/installing.html#installing-an-official-release), [imageio](https://imageio.readthedocs.io/en/stable/installation.html), and [SciPy](https://scipy.org/install.html).

### Table of contents

1. Examine an X-ray with `imageio`
2. Combine images with `np.stack()` to demonstrate progression
3. Edge detection using the Sobel filter and the Pythagorean theorem with `np.sqrt()` and `np.square()`
4. Apply masks to X-rays with `np.where()`

## Examine an X-ray with `imageio`

Let's begin with a simple example using just one X-ray image from the ChestX-ray8 dataset. 

The file — `00000011_001.png` — has been downloaded for you and saved in the `/biomedical-images` folder. 

> If you want to use your own samples, you can use [this one](https://openi.nlm.nih.gov/detailedresult?img=CXR3666_IM-1824-1001&query=chest%20infection&it=xg&req=4&npos=32) or search for various images on the [_Openi_](https://openi.nlm.nih.gov) database. Openi contains many biomedical images and it can be especially helpful if you have low bandwidth and/or are restricted by the amount of data you can download.


1. Read the image with `imageio`:

In [ ]:
import imageio

xray_image = imageio.imread('biomedical-images/00000011_001.png')

2. Check that its shape is 1024x1024 and that the array is made up of 8-bit integers:

In [ ]:
print(xray_image.shape)
print(xray_image.dtype)

3. Import `matplotlib` and display the image in a grayscale colormap:

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(xray_image, cmap='gray')
plt.axis('off')
plt.show()

## Combine images with `np.stack()` to demonstrate progression

With NumPy's `np.stack()` you can combine multiple X-rays to make an n-dimensional array and then show the "health progress" in a sequential manner.

In the next example, instead of 1 image you'll use 8 X-ray 1024x1024-pixel images from the ChestX-ray8 dataset that have been downloaded and extracted from one of the dataset files. They are numbered from `...000.png` to `...008.png` and let's assume they belong to the same patient.

1. Import NumPy and read in each of the X-rays:

In [ ]:
import numpy as np

file1 = imageio.imread('biomedical-images/00000011_000.png')
file2 = imageio.imread('biomedical-images/00000011_001.png')
file3 = imageio.imread('biomedical-images/00000011_003.png')
file4 = imageio.imread('biomedical-images/00000011_004.png')
file5 = imageio.imread('biomedical-images/00000011_005.png')
file6 = imageio.imread('biomedical-images/00000011_006.png')
file7 = imageio.imread('biomedical-images/00000011_007.png')
file8 = imageio.imread('biomedical-images/00000011_008.png')

combined_xray_images_1 = np.stack([file1, 
                                   file2, 
                                   file3, 
                                   file4, 
                                   file5, 
                                   file6, 
                                   file7, 
                                   file8])

Alternatively, you can append the image arrays as follows:

In [ ]:
combined_xray_images_2 = []

for i in range(8):
    single_xray_image = imageio.imread('biomedical-images/'+'00000011_00'+str(i)+'.png')
    combined_xray_images_2.append(single_xray_image)

2. Check the shape of the new X-ray image array containing 8 stacked images:

In [ ]:
combined_xray_images_1.shape

3. You can now display the "health progress" by plotting each of frames next to each other using matplotlib:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(30,30))

for i in range(8):
    x = combined_xray_images_1[i]
    axes[i].imshow(x, cmap='gray')
    axes[i].axis('off')

4. In addition, it can be helpful to show the progress as an animation. Let's create a GIF file with imageio and display the result in the notebook:

In [ ]:
from IPython.display import HTML

imageio.mimwrite('biomedical-images/xray_image.gif', combined_xray_images_1, format= '.gif', fps = 1) 

HTML('<img src="biomedical-images/xray_image.gif" width="200", hspace="20" vspace="20">')

## Edge detection using the Sobel filter and the Pythagorean theorem with `np.sqrt()` and `np.square()`

When processing  biomedical data, it can be useful to emphasize the "edges" to focus on particular features. 

To obtain the edge maps along the horizontal and vertical axes of an X-ray image, you can use the so-called [Sobel-Feldman operator (Sober filter)](https://en.wikipedia.org/wiki/Sobel_operator) technique and combine it with the [Pythagorean theorem](https://en.wikipedia.org/wiki/Pythagorean_theorem).

1. Import the `ndimage` module from SciPy, use the Sobel filters (`ndimage.sobel()`) on x- and y-axes, and calculate the distance between `x` and `y` (with the Sobel filters applied to them) using the Pythagorean theorem and NumPy (`np.sqrt(np.square(x) + np.square(y))`):

In [ ]:
from scipy import ndimage

x = ndimage.sobel(xray_image, axis=0)
y = ndimage.sobel(xray_image, axis=1)
squared_x = np.square(x)
squared_y = np.square(y)
sum_x_y = squared_x + squared_y 

xray_image_sobel_filter = np.sqrt(sum_x_y)

2. Check the data type of the new image array and change it to 32-bit floating points to make it work with matplotlib:

In [ ]:
print('The data type - before: ', xray_image_sobel_filter.dtype)

xray_image_sobel_filter_float32 = xray_image_sobel_filter.astype('float32')

print('The data type - after: ', xray_image_sobel_filter_float32.dtype)


3. Display the original X-ray and the one with the "edge" filter applied. Note that both the grayscale and `CMRmap` colormaps are used to help emphasize the edges:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,15))

axes[0].set_title('Original')
axes[0].imshow(xray_image, cmap='gray')
axes[1].set_title('Edges (grayscale)')
axes[1].imshow(xray_image_sobel_filter_float32, cmap='gray')
axes[2].set_title('Edges (hot)')
axes[2].imshow(xray_image_sobel_filter_float32, cmap='CMRmap')
for i in axes:
    i.axis('off')
plt.show()

## Apply masks to X-rays with `np.where()`

To screen out only certain pixels in X-ray images to help detect particular features, you can apply masks with NumPy's `np.where(condition, x, y)`.

1. Check that the X-ray image uses all 256 colors (from `0` to `255`):

In [ ]:
print(xray_image.min())
print(xray_image.max())

2. Visualize the pixel intensity distribution of the original raw X-ray image with `ndimage.histogram()` and matplotlib:

In [ ]:
pixel_intensity_distribution = ndimage.histogram(xray_image, min=xray_image.min(), max=xray_image.max(), bins=256)

plt.plot(pixel_intensity_distribution)
plt.title('Pixel intensity distribution')
plt.show()

As the pixel intensity distribution suggests, there are many low (between around 0 and 20) to and very high (between around 200 and 240) pixel values. 

3. You can create different conditional masks with NumPy's `np.where()` — for example, let's have only those values of the image with the pixels exceeding a certain threshold (and 0 otherwise):

In [ ]:
xray_image_mask_noisy = np.where(xray_image > 150, xray_image, 0)

plt.imshow(xray_image_mask_noisy, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
xray_image_mask_less_noisy = np.where(xray_image > 150, 1, 0)

plt.imshow(xray_image_mask_less_noisy, cmap='gray')
plt.axis('off')
plt.show()

Let's display some of the results of processed X-ray images you've worked with so far:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,15))

axes[0].set_title('Original')
axes[0].imshow(xray_image, cmap='gray')
axes[1].set_title('Edges')
axes[1].imshow(xray_image_sobel_filter_float32, cmap='gray')
axes[2].set_title('Mask (> 150, noisy)')
axes[2].imshow(xray_image_mask_noisy, cmap='gray')
axes[3].set_title('Mask (> 150), less noisy')
axes[3].imshow(xray_image_mask_less_noisy, cmap='gray')
for i in axes:
    i.axis('off')
plt.show()